In [ ]:
# imports

import os
import os.path
import cv2
import glob
import h5py
import tqdm
import argparse
import logging
from PIL import Image 

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_theme()
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms

import data, utils, models

In [ ]:
# Motion Compensation and FastDVDnet model definitions take from these official github repositories
# https://github.com/m-tassano/dvdnet
# https://github.com/m-tassano/fastdvdnet

"""
Functions to estimate the flow between two images and compensate it.
@author: Matias Tassano <mtassano@parisdescartes.fr>
"""
import numpy as np
import cv2

# Parameters of the motion estimation algorithms
def warp_flow(img, flow):
    '''
        Applies to img the transformation described by flow.
    '''
    assert len(flow.shape) == 3 and flow.shape[-1] == 2

    hf, wf = flow.shape[:2]
    # flow 		= -flow
    flow[:, :, 0] += np.arange(wf)
    flow[:, :, 1] += np.arange(hf)[:, np.newaxis]
    res = cv2.remap(img, flow, None, cv2.INTER_LINEAR)
    return res

def estimate_invflow(img0, img1, me_algo):
    '''
        Estimates inverse optical flow by using the me_algo algorithm.
    '''
    # # # img0, img1 have to be uint8 grayscale
    assert img0.dtype == 'uint8' and img1.dtype == 'uint8'

    # Create estimator object
    if me_algo == "DeepFlow":
        of_estim = cv2.optflow.createOptFlow_DeepFlow()
    elif me_algo == "SimpleFlow":
        of_estim = cv2.optflow.createOptFlow_SimpleFlow()
    elif me_algo == "TVL1":
        of_estim = cv2.DualTVL1OpticalFlow_create()
    else:
        raise Exception("Incorrect motion estimation algorithm")

    # Run flow estimation (inverse flow)
    flow = of_estim.calc(img1, img0, None)
    # flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    return flow

def align_frames(img_to_align, img_source, mc_alg='DeepFlow'):
    '''
        Applies to img_to_align a transformation which converts it into img_source.
        Args:
            img_to_align: HxWxC image
            img_source: HxWxC image
            mc_alg: selects between DeepFlow, SimpleFlow, and TVL1. DeepFlow runs by default.
        Returns:
            HxWxC aligned image
    '''

    # make sure images are uint8 in the [0, 255] range
    if img_source.max() <= 1.0:
        img_source = (img_source*255).clip(0, 255)
    img_source = img_source.astype(np.uint8)
    if img_to_align.max() <= 1.0:
        img_to_align = (img_to_align*255).clip(0, 255)
    img_to_align = img_to_align.astype(np.uint8)

    img0 = img_to_align[:, :, 0]
    img1 = img_source[:, :, 0]
    out_img = None

    # Align frames according to selection in mc_alg
    flow = estimate_invflow(img0, img1, mc_alg)

    # rectifier
    out_img = warp_flow(img_to_align, flow)

    return out_img

"""
Definition of the FastDVDnet model
Copyright (C) 2019, Matias Tassano <matias.tassano@parisdescartes.fr>
This program is free software: you can use, modify and/or
redistribute it under the terms of the GNU General Public
License as published by the Free Software Foundation, either
version 3 of the License, or (at your option) any later
version. You should have received a copy of this license along
this program. If not, see <http://www.gnu.org/licenses/>.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F

class CvBlock(nn.Module):
    '''(Conv2d => BN => ReLU) x 2'''
    def __init__(self, in_ch, out_ch):
        super(CvBlock, self).__init__()
        self.convblock = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.convblock(x)

class InputCvBlock(nn.Module):
    '''(Conv with num_in_frames groups => BN => ReLU) + (Conv => BN => ReLU)'''
    def __init__(self, num_in_frames, out_ch):
        super(InputCvBlock, self).__init__()
        self.interm_ch = 30
        self.convblock = nn.Sequential(
            nn.Conv2d(num_in_frames*(3+1), num_in_frames*self.interm_ch, \
                      kernel_size=3, padding=1, groups=num_in_frames, bias=False),
            nn.BatchNorm2d(num_in_frames*self.interm_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_in_frames*self.interm_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.convblock(x)

class DownBlock(nn.Module):
    '''Downscale + (Conv2d => BN => ReLU)*2'''
    def __init__(self, in_ch, out_ch):
        super(DownBlock, self).__init__()
        self.convblock = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, stride=2, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            CvBlock(out_ch, out_ch)
        )

    def forward(self, x):
        return self.convblock(x)

class UpBlock(nn.Module):
    '''(Conv2d => BN => ReLU)*2 + Upscale'''
    def __init__(self, in_ch, out_ch):
        super(UpBlock, self).__init__()
        self.convblock = nn.Sequential(
            CvBlock(in_ch, in_ch),
            nn.Conv2d(in_ch, out_ch*4, kernel_size=3, padding=1, bias=False),
            nn.PixelShuffle(2)
        )

    def forward(self, x):
        return self.convblock(x)

class OutputCvBlock(nn.Module):
    '''Conv2d => BN => ReLU => Conv2d'''
    def __init__(self, in_ch, out_ch):
        super(OutputCvBlock, self).__init__()
        self.convblock = nn.Sequential(
            nn.Conv2d(in_ch, in_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(in_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False)
        )

    def forward(self, x):
        return self.convblock(x)

class DenBlock(nn.Module):
    """ Definition of the denosing block of FastDVDnet.
    Inputs of constructor:
        num_input_frames: int. number of input frames
    Inputs of forward():
        xn: input frames of dim [N, C, H, W], (C=3 RGB)
        noise_map: array with noise map of dim [N, 1, H, W]
    """

    def __init__(self, num_input_frames=3):
        super(DenBlock, self).__init__()
        self.chs_lyr0 = 32
        self.chs_lyr1 = 64
        self.chs_lyr2 = 128

        self.inc = InputCvBlock(num_in_frames=num_input_frames, out_ch=self.chs_lyr0)
        self.downc0 = DownBlock(in_ch=self.chs_lyr0, out_ch=self.chs_lyr1)
        self.downc1 = DownBlock(in_ch=self.chs_lyr1, out_ch=self.chs_lyr2)
        self.upc2 = UpBlock(in_ch=self.chs_lyr2, out_ch=self.chs_lyr1)
        self.upc1 = UpBlock(in_ch=self.chs_lyr1, out_ch=self.chs_lyr0)
        self.outc = OutputCvBlock(in_ch=self.chs_lyr0, out_ch=3)

        self.reset_params()

    @staticmethod
    def weight_init(m):
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def reset_params(self):
        for _, m in enumerate(self.modules()):
            self.weight_init(m)

    def forward(self, in0, in1, in2, noise_map):
        '''Args:
            inX: Tensor, [N, C, H, W] in the [0., 1.] range
            noise_map: Tensor [N, 1, H, W] in the [0., 1.] range
        '''
        # Input convolution block
        x0 = self.inc(torch.cat((in0, noise_map, in1, noise_map, in2, noise_map), dim=1))
        # Downsampling
        x1 = self.downc0(x0)
        x2 = self.downc1(x1)
        # Upsampling
        x2 = self.upc2(x2)
        x1 = self.upc1(x1+x2)
        # Estimation
        x = self.outc(x0+x1)

        # Residual
        x = in1 - x

        return x

class FastDVDnet(nn.Module):
    """ Definition of the FastDVDnet model.
    Inputs of forward():
        xn: input frames of dim [N, C, H, W], (C=3 RGB)
        noise_map: array with noise map of dim [N, 1, H, W]
    """

    def __init__(self, num_input_frames=5):
        super(FastDVDnet, self).__init__()
        self.num_input_frames = num_input_frames
        # Define models of each denoising stage
        self.temp1 = DenBlock(num_input_frames=3)
        self.temp2 = DenBlock(num_input_frames=3)
        # Init weights
        self.reset_params()

    @staticmethod
    def weight_init(m):
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def reset_params(self):
        for _, m in enumerate(self.modules()):
            self.weight_init(m)

    def forward(self, x, noise_map):
        '''Args:
            x: Tensor, [N, num_frames*C, H, W] in the [0., 1.] range
            noise_map: Tensor [N, 1, H, W] in the [0., 1.] range
        '''
        # hack
        N, C, H, W = x.shape
        if(H%4 != 0):
            x = F.pad(x, [0, 0, 4-(H%4), 0], mode = 'reflect')
            noise_map = F.pad(noise_map, [0, 0, 4-(H%4), 0], mode = 'reflect')
        if(W%4 != 0):
            x = F.pad(x, [4-(W%4), 0, 0, 0], mode = 'reflect')
            noise_map = F.pad(noise_map, [4-(W%4), 0, 0, 0], mode = 'reflect')
        
        # Unpack inputs
        (x0, x1, x2, x3, x4) = tuple(x[:, 3*m:3*m+3, :, :] for m in range(self.num_input_frames))

        # First stage
        self.f1 = x20 = self.temp1(x0, x1, x2, noise_map)
        self.f2 = x21 = self.temp1(x1, x2, x3, noise_map)
        self.f3 = x22 = self.temp1(x2, x3, x4, noise_map)

        #Second stage
        x = self.temp2(x20, x21, x22, noise_map)
        
        # unhack
        N1, C1, H1, W1 = x.shape
        if(H%4 != 0):
            x = x[:, :, (4-(H%4)):H1, 0:W1]
        if(W%4 != 0):
            x = x[:, :, 0:H, (4-(W%4)):W1]

        return x

In [ ]:
# definition for loading model from a pretrained network file

def load_model(PATH, Fast=False, parallel=False, pretrained=True, old=True, load_opt=False):
    if not Fast:
        state_dict = torch.load(PATH, map_location="cpu")
        args = argparse.Namespace(**{**vars(state_dict["args"])})
        # ignore this
        if old:
            vars(args)['blind_noise'] = False

        model = models.build_model(args)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    else:
        model = FastDVDnet()
    
    if load_opt:
        for o, state in zip([optimizer], state_dict["optimizer"]):
            o.load_state_dict(state)
    
    if pretrained:
        if Fast:
            state_dict = torch.load(PATH)
        else:
            state_dict = torch.load(PATH)["model"][0]
        own_state = model.state_dict()
        
        for name, param in state_dict.items():
            if parallel:
                name = name[7:]
            if Fast:
                name = name.split('.', 1)[1]
            if name not in own_state:
                print("here", name)
                continue
            if isinstance(param, nn.Parameter):
                # backwards compatibility for serialized parameters
                param = param.data
            own_state[name].copy_(param)
        
    if not Fast:
        return model, optimizer, args
    else:
        return model

In [ ]:
# EXAMPLE NUMBER
example = 0 # Set 1 for another example

In [ ]:
# necessary variable deifnitons

parallel = True
Fast = False
pretrained = True
old = True
load_opt = False

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
transform = transforms.Compose([transforms.ToPILImage()])
to_gray = transforms.Compose([transforms.ToPILImage(), transforms.Grayscale(num_output_channels=1)])

dataset = "DAVIS"
if example == 0:
    video = "giant-slalom"
elif example == 1:
    video = "bus"
patch_size = 128
stride = 64
is_image = False
n_frames = 5
cpf = 3
mid = n_frames // 2
is_real = False

aug = 0

dist = 'G'
mode = 'S'
# change noise_std parameter here to produce results at various noise levels
noise_std = 30
min_noise = 0
max_noise = 100

batch_size = 1
lr = 1e-4

In [ ]:
# load model - UDVD

PATH = "../pretrained_models/blind_video_net.pt"

model, optimizer, args = load_model(PATH, parallel=parallel, pretrained=pretrained, old=old, load_opt=load_opt)
model.to(device)
print(model)

In [ ]:
# data loader

PATH = os.path.join("../datasets", dataset)

train_loader, test_loader = data.build_dataset("SingleVideo", PATH, batch_size=batch_size, dataset=dataset, video=video, image_size=patch_size, stride=stride, n_frames=n_frames, 
                                               aug=aug, dist=dist, mode=mode, noise_std=noise_std, min_noise=min_noise, max_noise=max_noise,
                                               sample=False)

In [ ]:
# prepare frame

px = 20*np.arange(0,6)+8
py = 20*np.arange(0,6)+8
ps, qs = np.meshgrid(px, py)
ps = ps.reshape(-1)
qs = qs.reshape(-1)

if example == 0:
    # giant-slalom
    num = 109
    x = 425; y = 115; w = 128; h = 128
    x1 = 38; y1 = 15; w1 = 64; h1 = 64
    ps = np.append(ps, 70); qs = np.append(qs, 47)
elif example == 1:
    # bus
    num = 25 
    x = 481; y = 219; w = 128; h = 128
    x1 = 0; y1 = 0; w1 = 128; h1 = 128
    ps = np.append(ps, 64); qs = np.append(qs, 64)

span = 1

sample = test_loader.dataset[num][0].unsqueeze(0)[:,:,y:y+h, x:x+w].to(device)
N, C, H, W = sample.shape

fixed_noises = []
for i in range(span+5):
    fixed_noises.append(utils.get_noise(sample[:,0:3,:,:], dist = "G", mode = 'S', noise_std = 255))

In [ ]:
# compute the jacobian (filters)

grad_mapss = []
pss = []; qss = [];

model.eval()

for l in range(len(ps)):
    grad_maps = []
    p = ps[l]
    q = qs[l]
    pss.append([ps[l]])
    qss.append([qs[l]])
    for i in range(span):
        sample = test_loader.dataset[num+i][0].unsqueeze(0)[:,:,y:y+h, x:x+w].to(device)
        
        clean_image = sample[:, (mid*cpf):((mid+1)*cpf), :, :]
        if not is_real:
            noise = (noise_std/255.0)*torch.cat(fixed_noises[i:i+5], 1)
            noisy_inputs = noise + sample
            noisy_frame = noisy_inputs[:, (mid*cpf):((mid+1)*cpf), :, :]
        else:
            noisy_inputs = sample
            noisy_frame = clean_image
        noisy_inputs = noisy_inputs.requires_grad_(True)
        
        N, C, H, W = sample.shape
        noise_map = (noise_std/255)*torch.ones(N, 1, H, W).to(device)

        if not Fast:
            output, _ = model(noisy_inputs)

            if not is_real:
                output, mean_image = utils.post_process(output, noisy_frame, model = "blind-video-net", sigma = noise_std/255, device=device)
                psnr = utils.psnr(clean_image, output)
        else:
            output = model(noisy_inputs, noise_map)
            psnr = utils.psnr(clean_image, output)

        loss = 100*output[:,:,q,p].mean()

        model.zero_grad()

        loss.backward()

        grads = noisy_inputs.grad.cpu().detach()

        grad_maps.append(grads)

        img = (np.sum(grads[0,9:12,:,:].cpu().detach().numpy(), axis=0)/cpf).reshape(H, W)

        ptile = 0.5*np.max(img)
        w_sum = 0
        p_final = 0
        q_final = 0
        for j in range(h):
            for k in range(w):
                if img[j][k] >= ptile:
                    p_final += k*img[j][k]
                    q_final += j*img[j][k]
                    w_sum += img[j][k]
        p_final /= w_sum
        q_final /= w_sum
        
        pss[l].append(int(p_final))
        qss[l].append(int(q_final))

        p = int(p_final)
        q = int(q_final)
        
    grad_mapss.append(grad_maps)
    
original_ps = np.copy(ps)
original_qs = np.copy(qs)

In [ ]:
# Gradients - Jacobian (filters)

save = 0
ps = original_ps[36:37]
qs = original_qs[36:37]

alpha = 1
limit = 0
for l in range(len(ps)):
    limit = max(limit, 1*np.max(np.abs(grad_mapss[l][0].numpy())))

for i in range(5):
    
    fig, ax = plt.subplots(figsize=(3,3))
    
    ax.hlines(h1/2, xmin=0, xmax=w1, linestyles='dashed', color='gray')
    ax.vlines(w1/2, ymin=0, ymax=h1, linestyles='dashed', color='gray')

    ax.imshow(np.sum(grad_mapss[-1][0][0,i*3:(i+1)*3, y1:y1+h1, x1:x1+w1].numpy(), axis=0).reshape(w1,h1), cmap="seismic",
              vmin=-limit, vmax=limit)

    rect = patches.Rectangle((0,0),w1-1,h1-1,linewidth=2,edgecolor="midnightblue" ,facecolor='none')
    ax.add_patch(rect)

    if i < 2:
        ax.set_title(f"Gradients at t-{2-i}")
    elif i == 2:
        ax.set_title("Gradients at t")
    else:
        ax.set_title(f"Gradients at t+{i-2}")
    ax.axis("off")
    
    ax.set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
                hspace = 0, wspace = 0)
    plt.margins(0,0)
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())
    
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    
plt.figure(figsize = (3,3))
img = plt.imshow(np.sum(grad_mapss[-1][0][0,6:9, y1:y1+h1, x1:x1+w1].numpy(), axis=0).reshape(w1,h1), 
                 cmap="seismic", vmin=-limit, vmax=limit)
plt.xticks([])
plt.yticks([])
plt.tight_layout()

fig, ax = plt.subplots(figsize=(3,3))
cbar = plt.colorbar(img, ax=ax)
ax.remove()
plt.tight_layout()

In [ ]:
# Images

for i in range(5):
    pclr = 'lime'
    
    fig, ax = plt.subplots(figsize=(3,3), frameon=False)
    ax.hlines(h1/2, xmin=0, xmax=w1, linestyles='dashed', color='white')
    ax.vlines(w1/2, ymin=0, ymax=h1, linestyles='dashed', color='white')
    ax.imshow(transform(test_loader.dataset[num][0][i*3:(i+1)*3, y+y1:y+y1+h1, x+x1:x+x1+w1]))

    rect = patches.Rectangle((0,0),w1-1,h1-1,linewidth=2,edgecolor="midnightblue" ,facecolor='none')
    ax.add_patch(rect)
        
    if i < 2:
        ax.set_title(f"True frame at t-{2-i}")
    elif i == 2:
        ax.set_title("True frame at t")
    else:
        ax.set_title(f"True frame at t+{i-2}")
    ax.axis("off")
    
    ax.set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
                hspace = 0, wspace = 0)
    plt.margins(0,0)
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())
    
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    

fig, ax = plt.subplots(figsize=(3,3), frameon=False)

ax.imshow(transform(test_loader.dataset[num][0][6:9, :, :]))

rect = patches.Rectangle((x+x1,y+y1),w1,h1,linewidth=1,edgecolor='midnightblue',facecolor='none')
ax.add_patch(rect)

ax.set_title("Complete frame and crop")
ax.axis("off")

ax.set_axis_off()
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
plt.margins(0,0)
ax.xaxis.set_major_locator(plt.NullLocator())
ax.yaxis.set_major_locator(plt.NullLocator())

plt.xticks([])
plt.yticks([])
plt.tight_layout()


fig, ax = plt.subplots(figsize=(3,3), frameon=False)

noisy_img = np.array(transform(test_loader.dataset[num][0][6:9, y+y1:y+y1+h1, x+x1:x+x1+w1]
                               + (noise_std/255)*fixed_noises[2][0,:, y1:y1+h1, x1:x1+w1].cpu()))

ax.imshow(transform(noisy_img))
rect = patches.Rectangle((0,0),w1-1,h1-1,linewidth=2,edgecolor='midnightblue',facecolor='none')
ax.add_patch(rect)

ax.set_title("Noisy frame")
ax.axis("off")

ax.set_axis_off()
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
plt.margins(0,0)
ax.xaxis.set_major_locator(plt.NullLocator())
ax.yaxis.set_major_locator(plt.NullLocator())

plt.xticks([])
plt.yticks([])
plt.tight_layout()


fig, ax = plt.subplots(figsize=(3,3), frameon=False)

ax.hlines(h1/2, xmin=0, xmax=w1, linestyles='dashed', color='white')
ax.vlines(w1/2, ymin=0, ymax=h1, linestyles='dashed', color='white')

ax.imshow(transform(output[0,:,y1:y1+h1,x1:x1+w1].cpu().detach()))

rect = patches.Rectangle((0,0),w1-1,h1-1,linewidth=2,edgecolor='midnightblue',facecolor='none')
ax.add_patch(rect)

ax.set_title("Denoised frame")
ax.axis("off")

ax.set_axis_off()
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
plt.margins(0,0)
ax.xaxis.set_major_locator(plt.NullLocator())
ax.yaxis.set_major_locator(plt.NullLocator())

plt.xticks([])
plt.yticks([])
plt.tight_layout()

In [ ]:
# Calulate the arrows for Motion Compensation

ps = original_ps[0:36]
qs = original_qs[0:36]

limit = 0.4
fpss = []; fqss = []; npss = []; nqss = []
for l in range(len(ps)):
    fpss.append([ps[l]])
    npss.append([ps[l]])
    fqss.append([qs[l]])
    nqss.append([qs[l]])

for i in range(span+1):
    img = np.array(transform(test_loader.dataset[num+i][0][6:9, y:y+h, x:x+w]))
    noisy_img = np.array(transform(test_loader.dataset[num+i][0][6:9, y:y+h, x:x+w] + (noise_std/255)*fixed_noises[2+i][0,:,:,:].cpu()))

    img0 = np.array(to_gray(test_loader.dataset[num+i][0][6:9, y:y+h, x:x+w]))
    img1 = np.array(to_gray(test_loader.dataset[num+i][0][9:12, y:y+h, x:x+w]))
    noisy_img0 = np.array(to_gray(test_loader.dataset[num+i][0][6:9, y:y+h, x:x+w] + (noise_std/255)*fixed_noises[2+i][0,:,:,:].cpu()))
    noisy_img1 = np.array(to_gray(test_loader.dataset[num+i][0][9:12, y:y+h, x:x+w] + (noise_std/255)*fixed_noises[3+i][0,:,:,:].cpu()))
    
    flow = estimate_invflow(img0, img1, "DeepFlow")
    noisy_flow = estimate_invflow(noisy_img0, noisy_img1, "DeepFlow")
    for l in range(len(ps)):
        p = fpss[l][-1]; q = fqss[l][-1]; Np = npss[l][-1]; Nq = nqss[l][-1]
        fpss[l].append(min(p - int(flow[q][p][0]), w-1)); 
        fqss[l].append(min(q - int(flow[q][p][1]), h-1));
        npss[l].append(min(Np - int(noisy_flow[Nq][Np][0]), w-1)); 
        nqss[l].append(min(Nq - int(noisy_flow[Nq][Np][1]), h-1));

In [ ]:
# Motion Compensation

scale = 2
width = 2
head_width = 3*width
color = "darkorange"

img = np.array(transform(test_loader.dataset[num][0][6:9, y:y+h, x:x+w]))
noisy_img = np.array(transform(test_loader.dataset[num][0][6:9, y:y+h, x:x+w] + (noise_std/255)*fixed_noises[2][0,:,:,:].cpu()))

fig, ax = plt.subplots(figsize=(4,4), frameon=False)
    
ax.imshow(noisy_img)
ax.set_title("Noisy frame")
ax.axis("off")

plt.tight_layout()


fig, ax = plt.subplots(figsize=(4,4), frameon=False)

ax.imshow(img)
ax.set_title("DeepFlow on clean video")
ax.axis("off")

for l in range(len(ps)):
    ax.arrow(fpss[l][0], fqss[l][0], 
             scale*(fpss[l][1]-fpss[l][0]), 
             scale*(fqss[l][1]-fqss[l][0]), width=width, head_width=head_width,
             length_includes_head=False, facecolor=color, edgecolor='midnightblue')
    
plt.tight_layout()
    
    
fig, ax = plt.subplots(figsize=(4,4), frameon=False)

ax.imshow(img)
ax.set_title("UDVD implicit motion estimation")
ax.axis("off")

for l in range(len(ps)):
    ax.arrow(pss[l][0], qss[l][0], 
             scale*(pss[l][1]-pss[l][0]), 
             scale*(qss[l][1]-qss[l][0]), width=width, head_width=head_width,
             length_includes_head=False, facecolor=color, edgecolor='midnightblue')
    
plt.tight_layout()


fig, ax = plt.subplots(figsize=(4,4), frameon=False)

ax.imshow(img)
ax.set_title("DeepFlow on noisy video")
ax.axis("off")

for l in range(len(ps)):
    ax.arrow(npss[l][0], nqss[l][0], 
             scale*(npss[l][1]-npss[l][0]), 
             scale*(nqss[l][1]-nqss[l][0]), width=width, head_width=head_width,
             length_includes_head=False, facecolor=color, edgecolor='midnightblue')
    
plt.tight_layout()